## Code to find intersections of ICESat-2 and ATM data

**by Allison Chartrand**

**June 2019 ICESat-2 Hackweek**


In [72]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
from scipy.spatial import distance

In [73]:
%matplotlib widget

**Import Data**

Get ATL06 data and put into dataframe

In [80]:
ATL06filename = '~/xtrak/data_prod/ZachISatData_wSmoooth.csv'
ATMfilename = '~/xtrak/data_prod/ATMproof_20140429_wSmooth_ac.csv'

OutputFilename = '~/xtrak/data_prod/InterX_ATM2014_AllSmooth.csv'

In [38]:
ATL06data = pd.read_csv(ATL06filename,parse_dates=[4])
ATL06data.head()

,lon,lat,h,track,date,x,y,hSm,hSupSm
0,-22.520585,78.700068,579.33765,gt3r,2019-01-04 12:24:25,469494.782922,-1.134613e+06,NaN,NaN
1,-22.520755,78.700244,579.03015,gt3r,2019-01-04 12:24:25,469484.057930,-1.134597e+06,NaN,NaN
2,-22.520925,78.700420,578.78815,gt3r,2019-01-04 12:24:25,469473.331404,-1.134581e+06,NaN,NaN
3,-22.521094,78.700596,578.57214,gt3r,2019-01-04 12:24:25,469462.614457,-1.134564e+06,NaN,NaN
4,-22.521264,78.700773,578.33830,gt3r,2019-01-04 12:24:25,469451.884907,-1.134548e+06,NaN,NaN


Get target points from ATM csv and put into dataframe

In [81]:
ATMdata = pd.read_csv(ATMfilename)

tpoints = ATMdata[['PS_x','PS_y']].copy()
tpoints = tpoints.values
ATMdata.head()

,Unnamed: 0,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along,slope_NS,slope_EW,ATM_elev_Sm,ATM_elev_SupSm
0,0,79.150302,335.663786,415944.317386,-1.102872e+06,883.1842,18036.512689,-0.006359,-0.009239,NaN,NaN
1,1,79.150190,335.665218,415976.200234,-1.102873e+06,883.0373,18068.412960,-0.005456,-0.006168,NaN,NaN
2,2,79.150078,335.666651,416008.102644,-1.102874e+06,882.8962,18100.332515,-0.003837,-0.006896,NaN,NaN
3,3,79.149967,335.668086,416040.005299,-1.102875e+06,882.7560,18132.248675,-0.004822,-0.005571,NaN,NaN
4,4,79.149855,335.669519,416071.908333,-1.102876e+06,882.5889,18164.168794,-0.003373,-0.009250,NaN,NaN


Define nearest neighbor algorithm

In [76]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    closest_dist = np.min(distance.cdist([node], nodes,'euclidean'))
    return closest_index, closest_dist

Make an empty dataframe for intersections
Get query points from ATL06 csv and put into dataframe
Loop through unique dates and ground tracks and find the closest point between each ground track and the flowline
Store intersection along-flow distance, z_ATM, z_ATL06 and other helpful data in Intersections dataframe

In [82]:
Intersections = {'dist_along':[],'ATM_elev':[],'idx_ATM':[],'z_ATL06':[],'t_ATL06':[],'idx_ATL06':[],'gt_ATL06':[]}
Intersections = pd.DataFrame(data = Intersections)
i = 0
for day in ATL06data.date.unique():

    for tr in ATL06data.track.unique():
        close_idx = []
        min_dist = []
        AddDatarow = []

        #dfTran is a single radar transect
        dfTran = ATL06data.query('date == @day & track == @tr')
        if dfTran.shape[0] == 0:
            continue
            
        
        qpoints = dfTran[['x','y']].copy()
        qpoints = qpoints.values

        
        for j in range(len(tpoints)):
            close_idxT,min_distT = closest_node(tpoints[j,:], qpoints)
            close_idx = np.append(close_idx,[close_idxT])
            min_dist = np.append(min_dist,[min_distT])
        if min(min_dist) > 1000:
            continue
        
        tpt_idx = np.argmin(min_dist)
        qpt_idx = int(close_idx[tpt_idx])
        dfTran_idx = dfTran.loc[dfTran['date'] == day].index[0]
        qpt_idx = qpt_idx+dfTran_idx
        
        AddDatarow = {'dist_along':[ATMdata.loc[tpt_idx,'dist_along']],
                      'ATM_elev':[ATMdata.loc[tpt_idx,'ATM_elev_SupSm']],
                      'idx_ATM':[tpt_idx],
                      'z_ATL06':[dfTran.loc[qpt_idx,'hSupSm']],
                      't_ATL06':[dfTran.loc[qpt_idx,'date']],
                      'idx_ATL06':[qpt_idx],
                      'gt_ATL06':[dfTran.loc[qpt_idx,'track']]}
        AddDatarow = pd.DataFrame(data = AddDatarow)
        Intersections = Intersections.append(AddDatarow,ignore_index=True)
#         Intersections.reset_index(drop=True)
        

In [83]:
Intersections.head()


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,125289.795232,38.49300,3213.0,35.501406,2019-02-15 10:09:55,1576.0,gt1l
1,125354.749926,38.22330,3215.0,34.084645,2019-02-15 10:09:55,4931.0,gt1r
2,96208.910678,90.80875,2367.0,69.272560,2019-01-16 01:09:49,39474.0,gt3r
3,102696.987249,27.22505,2566.0,28.789461,2019-01-16 01:09:49,19999.0,gt1l
4,102632.054028,27.14045,2564.0,29.224519,2019-01-16 01:09:49,23921.0,gt1r


Plot 

In [84]:
f, ax = plt.subplots()
plt.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],c=Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'], c = Time,s=1)
plt.plot(Intersections['dist_along']/1000,Intersections['ATM_elev'],'.')
# cb = f.colorbar()
plt.colorbar()

# cb.ax.set_yticklabels(df.index.strftime('%b %Y'))

FigureCanvasNbAgg()

In [97]:
plt.close('all')

Sort the data by date and track

In [85]:
IntersectionsSort = Intersections.sort_values(by=['t_ATL06', 'gt_ATL06'])
IntersectionsSort.reset_index()

,index,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,161,122581.170689,45.04805,3130.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l
1,162,122647.058967,44.76880,3132.0,42.156155,2018-10-18 15:53:52,621074.0,gt1r
2,163,125810.129706,36.67925,3229.0,34.448452,2018-10-18 15:53:52,624012.0,gt2l
3,164,125875.304455,36.28405,3231.0,32.827094,2018-10-18 15:53:52,626687.0,gt2r
4,52,80573.944294,295.06470,1901.0,NaN,2018-10-21 05:21:45,205271.0,gt2r
5,51,77393.293766,269.78040,1803.0,255.108995,2018-10-21 05:21:45,205803.0,gt3r
6,103,67655.902876,416.97825,1538.0,361.948000,2018-10-25 05:13:27,398282.0,gt1l
7,104,67687.240097,415.62490,1539.0,391.111190,2018-10-25 05:13:27,400497.0,gt1r
8,105,64607.360881,425.31270,1439.0,NaN,2018-10-25 05:13:27,401953.0,gt2l
9,106,64484.432996,425.31270,1435.0,410.575870,2018-10-25 05:13:27,403762.0,gt2r


Compute slope between adjacent intersections of beam pairs

In [86]:
InterX = IntersectionsSort.values
InterX = InterX

Dist = np.diff(IntersectionsSort['dist_along'].values)
Dist[Dist == 0] = float('NaN')
InterXslopeATL06 = np.diff(IntersectionsSort['z_ATL06'].values)/Dist
InterXslopeATM = np.diff(IntersectionsSort['ATM_elev'].values)/Dist

# i = 0
# for i in Dist():
InterXslopeATL06[np.abs(Dist) > 150] = float('NaN')
InterXslopeATL06 = np.append(InterXslopeATL06,[0])

InterXslopeATM[np.abs(Dist) > 150] = float('NaN')
InterXslopeATM = np.append(InterXslopeATM,[float('NaN')])


/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()
/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


In [87]:
IntersectionsSort['dist_along'].values

array([122581.17068851, 122647.058967  , 125810.12970578, 125875.30445519,
        80573.94429394,  77393.2937659 ,  67655.90287629,  67687.24009683,
        64607.36088144,  64484.43299646,  61379.68549351,  61315.68444139,
        89377.26506363,  89479.46460643,  92780.62748277,  92848.52269775,
        96176.15302596,  96241.6091144 , 121954.02897127, 121854.85171978,
       118653.4943702 , 118588.41551142, 115361.06654741, 115295.19138123,
       105596.02917436, 105001.33969727, 102145.2482091 , 102080.40453568,
        98960.11443049,  98894.60315521, 127917.08666673, 127917.08666673,
        89241.05814438,  89172.93531811,  86031.58538731,  85963.53630899,
        82864.46615351,  82798.52326247, 111746.23718542, 111812.54554675,
       114966.47851405, 115032.17091117, 118164.36081161, 118262.38014119,
        73136.80798326,  73070.49437217,  69782.28637025,  69782.28637025,
        66747.76440998,  66654.49722872,  95027.94786504,  95126.69047093,
        98401.58285999,  

Add 90m slope arrays back into IntersectionsSort and save to csv

In [70]:
IntersectionsSort = IntersectionsSort.assign(slope_ATM= InterXslopeATM)
IntersectionsSort = IntersectionsSort.assign(slope_ATL06 = InterXslopeATL06)
IntersectionsSort.head()

,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,slope_ATM,slope_ATL06
160,122622.959615,44.03935,3314.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l,-0.002130,-0.012068
161,122685.347575,43.90645,3316.0,42.016131,2018-10-18 15:53:52,621070.0,gt1r,NaN,NaN
162,125829.732109,35.31125,3416.0,34.973022,2018-10-18 15:53:52,624014.0,gt2l,-0.003442,-0.031405
163,125892.855741,35.09395,3418.0,32.990604,2018-10-18 15:53:52,626689.0,gt2r,NaN,NaN
51,80616.609191,275.81145,2136.0,NaN,2018-10-21 05:21:45,205271.0,gt2r,NaN,NaN


Save to file

In [88]:

IntersectionsSort.to_csv(OutputFilename,index=False)